In [1]:
import sys
sys.path.append('../')
from plotting import CandlePlot
import pandas as pd

In [2]:
df = pd.read_pickle('../data/GBP_JPY_H1.pkl')

In [3]:
df_an = df[['time', 'mid_o','mid_h', 'mid_l', 'mid_c']].copy()

In [4]:
df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
37219,2021-12-30 19:00:00+00:00,155.312,155.444,155.274,155.444
37220,2021-12-30 20:00:00+00:00,155.442,155.470,155.289,155.312
37221,2021-12-30 21:00:00+00:00,155.317,155.382,155.286,155.382
37222,2021-12-30 22:00:00+00:00,155.353,155.381,155.282,155.329
37223,2021-12-30 23:00:00+00:00,155.336,155.360,155.284,155.310


In [15]:
direction = df_an.mid_c - df_an.mid_o
body_size = abs(direction)
direction = [1 if x >= 0 else -1 for x in direction]
full_range = df_an.mid_h - df_an.mid_l
body_perc = (body_size / full_range) * 100
body_lower = df_an[['mid_c', 'mid_o']].min(axis=1)
body_upper = df_an[['mid_c', 'mid_o']].max(axis=1)


In [16]:
df_an['body_lower'] = body_lower
df_an['body_upper'] = body_upper

In [18]:
dfp = df_an.iloc[-40:]
cp = CandlePlot(dfp)
cp.show_plot(line_traces=['body_upper', 'body_lower'])